## An Automated Portfolio Trading System with Feature Preprocessing and Recurrent Reinforcement Learning

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.data import Data, INDICATORS, get_buy_and_hold_returns

from lib.model import RRL, train, validation

from lib.backtest import (
    run_rrl_strategy, 
    create_backtest_dataset, 
    generate_layout, 
    generate_nrows, 
    generate_subplot_titles, 
    init_subplots, 
    populate_subplots,
    plot_cumulative_profits, 
    generate_df_barplot, 
    make_cumrets_barplot,
)

from lib.metrics import (
    calc_cumulative_profits, 
    calc_sharpe_ratio, 
)

In [3]:
import numpy as np 
import pandas as pd 

import plotly.graph_objects as go
import plotly.express as px

from pickle import load, dump

from colour import Color

### Data

In [4]:
assets = ["XOM", "VZ", "NKE", "AMAT", "MCD", "MSFT", "AAP", "NOV"]
indicators = list(INDICATORS.keys())

start_date = "2009-12-31"
end_date = "2017-12-29"

In [5]:
data = Data(start_date, end_date, assets, indicators)
data

[*********************100%***********************]  8 of 8 completed


Data(start_date='2009-12-31', end_date='2017-12-29', assets=['XOM', 'VZ', 'NKE', 'AMAT', 'MCD', 'MSFT', 'AAP', 'NOV'], indicators=['MOM', 'MACD', 'MFI', 'RSI', 'ATR', 'NATR', 'CO', 'OBV'], window_size=100)

In [6]:
buy_and_hold_returns = get_buy_and_hold_returns(data.df)

### `RRL` - Training and validation

In [7]:
initial_invest = 100
n_epochs = 20

#### No transaction fees 

$\delta=0$

In [19]:
rrl = RRL(n_assets=data.n_assets, n_features=data.n_features) 
print(rrl) 

results_no_fees = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

with open("./backup/results_no_fees.pkl", "wb") as f: 
    dump(results_no_fees, f)

RRL(n_assets=8, n_features=8, delta=0, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  20%|██        | 4/20 [00:00<00:01, 13.69it/s, sharpe_ratio=0.357]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 2.240829495772644
Cumulative profits: 120.65850820091049
Training window: 2010-06-30/2010-10-14


Training in progress...:  40%|████      | 8/20 [00:00<00:00, 18.56it/s, sharpe_ratio=2.27]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 2.1490217436369385
Cumulative profits: 139.96064801732425
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -0.06318712163200128
Cumulative profits: 293.67427192338374
Training window: 2017-03-31/2017-07-18


Training in progress...: 100%|██████████| 20/20 [00:00<00:00, 23.25it/s, sharpe_ratio=-.558]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 2.327990186946607
Cumulative profits: 348.13236014135066
Training window: 2017-08-23/2017-12-07


Training in progress...:  55%|█████▌    | 11/20 [00:00<00:00, 13.91it/s, sharpe_ratio=2.48]

Less than 30 observations in the validation set.


#### Transactions fees 

$\delta \in \{0.001, 0.003, 0.005\}$

In [20]:
rrl = RRL(n_assets=data.n_assets, n_features=data.n_features, delta=.001) 
print(rrl)

results_10bps = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

with open("./backup/results_10bps.pkl", "wb") as f: 
    dump(results_10bps, f)

RRL(n_assets=8, n_features=8, delta=0.001, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  75%|███████▌  | 15/20 [00:01<00:00, 13.70it/s, sharpe_ratio=0.261]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 2.0809343395150592
Cumulative profits: 119.47381968292655
Training window: 2010-06-30/2010-10-14


Training in progress...:  70%|███████   | 14/20 [00:00<00:00, 17.30it/s, sharpe_ratio=1.99]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 1.6444521919131483
Cumulative profits: 132.8164845568626
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -1.3081807535521073
Cumulative profits: 173.777043777998
Training window: 2017-03-31/2017-07-18


Training in progress...: 100%|██████████| 20/20 [00:00<00:00, 23.05it/s, sharpe_ratio=-1.22]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 2.3089823729140475
Cumulative profits: 204.7768384835098
Training window: 2017-08-23/2017-12-07


Training in progress...:  65%|██████▌   | 13/20 [00:00<00:00, 16.69it/s, sharpe_ratio=2.16]

Less than 30 observations in the validation set.


In [21]:
rrl = RRL(n_assets=data.n_assets, n_features=data.n_features, delta=.003) 
print(rrl)

results_30bps = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

with open("./backup/results_30bps.pkl", "wb") as f: 
    dump(results_30bps, f)

RRL(n_assets=8, n_features=8, delta=0.003, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  55%|█████▌    | 11/20 [00:00<00:00, 16.11it/s, sharpe_ratio=0.0433]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 1.6787823664130732
Cumulative profits: 115.53158420592541
Training window: 2010-06-30/2010-10-14


Training in progress...:  75%|███████▌  | 15/20 [00:00<00:00, 18.88it/s, sharpe_ratio=1.43]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 1.4241667392589288
Cumulative profits: 126.99981843214958
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -2.588374001239558
Cumulative profits: 89.85383420639526
Training window: 2017-03-31/2017-07-18


Training in progress...:  35%|███▌      | 7/20 [00:00<00:00, 22.62it/s, sharpe_ratio=-2.4] 


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 0.9518880425180399
Cumulative profits: 95.39394069950994
Training window: 2017-08-23/2017-12-07


Training in progress...:  35%|███▌      | 7/20 [00:00<00:00, 17.38it/s, sharpe_ratio=0.805]

Less than 30 observations in the validation set.


In [22]:
rrl = RRL(n_assets=data.n_assets, n_features=data.n_features, delta=.005) 
print(rrl)

results_50bps = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

with open("./backup/results_50bps.pkl", "wb") as f: 
    dump(results_50bps, f)

RRL(n_assets=8, n_features=8, delta=0.005, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...: 100%|██████████| 20/20 [00:01<00:00, 17.88it/s, sharpe_ratio=0.13]  


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 1.3390172127648994
Cumulative profits: 112.02979280060705
Training window: 2010-06-30/2010-10-14


Training in progress...: 100%|██████████| 20/20 [00:01<00:00, 19.31it/s, sharpe_ratio=1.32]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 0.8079327738560962
Cumulative profits: 117.91290224415356
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -0.5077070096722939
Cumulative profits: 77.95949191450455
Training window: 2017-03-31/2017-07-18


Training in progress...:   5%|▌         | 1/20 [00:00<00:01,  9.91it/s, sharpe_ratio=-.508]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 0.5001823062092275
Cumulative profits: 80.26926913716666
Training window: 2017-08-23/2017-12-07


Training in progress...:   5%|▌         | 1/20 [00:00<00:02,  8.53it/s, sharpe_ratio=0.5]

Less than 30 observations in the validation set.


### Backtest

In [7]:
file_names = ["results_no_fees", "results_10bps", "results_30bps", "results_50bps"]

results = {
    f.replace("results_", ""): load(open(f"./backup/{f}.pkl", "rb"))
    for f in file_names
}

In [8]:
dfs_backtest = {
    k: create_backtest_dataset(v)
    for k, v in results.items()
}

#### Cumulative profits

In [10]:
initial_invest = 100

In [11]:
layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title="", 
    x_axis_args={"gridcolor": "lightgrey"}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "$"}}
)
fig = plot_cumulative_profits(
    dfs_backtest, 
    buy_and_hold_returns, 
    initial_invest, 
    layout)

fig.show()

#### Analysis per trading window for one setting

In [18]:
setting = "10bps"

df = dfs_backtest[setting]
batch_names = np.unique(df["Batch"])

In [19]:
metric ="Cumulative returns"
fig_title = f"{metric} per trading window | {setting}"

ncols = 4
nrows = generate_nrows(len(batch_names), ncols)

titles = generate_subplot_titles(batch_names)

fig = init_subplots(nrows, ncols, titles)
fig = populate_subplots(fig, df, batch_names, titles, ncols, y_varname=metric, initial_invest=initial_invest)

layout = generate_layout(
    fig_height=1200, 
    fig_width=1400, 
    fig_title=fig_title)
    
fig.update_layout(layout)

In [24]:
df_barplot = generate_df_barplot(df)
title = f"% Cumulative returns at the end of each trading window | {setting}"

layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title=title,
    x_axis_args={"title": {"text": "Trading window"}}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "%"}}
)
fig = make_cumrets_barplot(df_barplot, layout)

fig.show()